**GROUP MEMBERS:**
*   MAHA SOHAIL ANWER 19L-1089
*   AYESHA RIZWAN 19L-2308
*   RIVA NOUMAN KHAN 19L-1117
*   LAIBA ARSHAD 19L-1119

In [ ]:
import numpy as np
import pandas as pd
import random
import copy

def randomizeWeights(rowCount, colCount):
  weights = np.ones((rowCount, colCount))
  for i in range(rowCount):
    for j in range(colCount):
      weights[i][j] = round(np.random.randn(), 4)
  return weights
def make1D(w):
  weights = []
  for i in range(len(w)):
      nor = (len(w[i]))
      for j in range(nor): #loop for number of rows
        noc = (len(w[i][j]))
        for k in range(noc): #loop for number of cols
          weights.append(w[i][j][k])
  return weights
def make3D(w, inputs, outputs):
  weights = []
  ind = 0
  for i in range(2):
      layer = []
      nor = 0
      if i == 0:
        nor = inputs
      else:
        nor = outputs
      for j in range(nor): #loop for number of rows
        row = []
        noc = 0
        if i == 0:
          noc = outputs
        else:
          noc = 1
        for k in range(noc): #loop for number of cols
          if ind < inputs * outputs + outputs:
            row.append(w[ind])
          ind += 1
        layer.append(row)
      weights.append(layer)
  return weights
#activation functions
def step(z):
  if z >= 0:
    return 2
  else:
    return 1
def sigmoid(z):
  a = 1/(1 + np.exp(-z))
  return round(a, 4)
def tanh(z):
  a = (2/(1 + np.exp(-2 * z))) - 1
  return round(a, 4)
def sigmoidClassifier(a):
  if a > 0.5:
    return 2
  else:
    return 1
def tanhClassifier(a):
  if a > 0:
    return 2
  else:
    return 1
def getUserInput():
  print('Enter filename')
  filename = input()
  df = pd.read_csv(filename)
  noOfInputs = len(df.columns) - 1
  print('Enter no Of Hidden Nodes')
  noOfHiddenNodes = int(input())
  print('Enter Activation Function for Hidden Nodes')
  actFunc1 = input()
  print('Enter Activation Function for Output')
  actFunc2 = input()
  network = []
  network.append([noOfInputs, noOfHiddenNodes, actFunc1])
  network.append([noOfHiddenNodes, 1, actFunc2])
  inputs = []
  for i in range(len(df.index)):
    dataInputs = []
    for j in range(len(df.columns)):
      dataInputs.append(float(df.iloc[i][j]))
    inputs.append(dataInputs)
  return (network, inputs)

In [ ]:
class neuralNetwork:
  def __init__(self, network, inputsGiven): #parameters: network list, inputs and labels provided
    self.weights = [] #to store weights of each layer
    self.bias = [] #to store bias of each node in layer
    self.activationFunc = [] #activation function in hidden and output layer
    self.inputs = [] #list of inputs
    self.labels = [] #list of labels corresponding to inputs
    for i in range(len(inputsGiven)): #storing inputs and labels
      input = []
      for j in range(len(inputsGiven[i]) - 1):
        input.append(inputsGiven[i][j])
      self.inputs.append(input)
      self.labels.append(inputsGiven[i][len(inputsGiven[i]) - 1])
    for layer in network: #storing bias based on input and output sizes
      inputCount = layer[0]
      outputCount = layer[1]
      self.activationFunc.append(layer[2])
      self.bias.append(randomizeWeights(outputCount, 1))

  def perceptron(self, inputs, outputs, layer, n): #parameters: inputs array, no of outputs, layer number, perceptron number
    z = 0
    #calculation of z
    for i in range(len(inputs)):
      z = z + float(inputs[i]) * self.weights[layer][i][n]
    z = z + self.bias[layer][n][0]
    a = 0
    #calculation of a based on activation function
    if self.activationFunc[layer] == 'sigmoid':
      a = sigmoid(z)
    elif self.activationFunc[layer] == 'tanh':
      a = tanh(z)
    else:
      a = step(z)
    return a
  def predValue(self, sampleNo): #parameters: sample number
    newInputs = []
    outputs = ((np.array(self.weights[0])).shape)[1]
    for j in range(outputs):
      newInputs.append(self.perceptron(self.inputs[sampleNo], outputs, 0, j))
    outputs = ((np.array(self.weights[1])).shape)[1]
    pred = self.perceptron(newInputs, outputs, 1, 0)
    yhat = pred
    if self.activationFunc[1] == 'sigmoid':
      yhat = sigmoidClassifier(pred)
    elif self.activationFunc[1] == 'tanh':
      yhat = tanhClassifier(pred)
    return (yhat)
  def accuracy(self, sRate, eRate): #calculation of accuracy of entire network
    accuracyCount = 0
    totalCount = 0
    for i in range(int(len(self.inputs) * sRate), int(len(self.inputs) * eRate)):
      yhat = self.predValue(i)
      if yhat == self.labels[i]:
        accuracyCount += 1
      totalCount += 1
    return (accuracyCount / totalCount) * 100
  def setWeights(self, w):
    weights = []
    for i in range(len(w)):
      layer = []
      nor = (len(w[i]))
      for j in range(nor): #loop for number of rows
        row = []
        noc = (len(w[i][j]))
        for k in range(noc): #loop for number of cols
          row.append(w[i][j][k])
        layer.append(row)
      weights.append(layer)
    self.weights = weights


In [ ]:
class chromosome:
  def __init__(self, weights, nn, inputCount, outputCount, tr): #weights in 1D
    self.chr = weights
    self.fitness = self.fitnessFunc(nn, inputCount, outputCount, tr)
    
  def fitnessFunc(self, nn, inputCount, outputCount, tr): #calculating fitness function based on accuracy
    w = make3D(self.chr, inputCount, outputCount)
    nn.setWeights(w)
    acc = nn.accuracy(0, tr)
    return (acc)
  
def generateChromosome(nn, inputCount, outputCount, tr):
  ch = []
  for i in range(inputCount * outputCount + outputCount):
    ch.append(round(np.random.randn(), 4))
  chromo = chromosome(ch, nn, inputCount, outputCount, tr)
  return chromo

class geneticAlgorithm:
  def __init__(self, tRate):
    self.pop = []
    self.pop_size = 5
    self.selectionRate = 0.2
    self.crossoverRate = 0.4
    self.mutationRate = 0.4
    self.trainingRate = tRate
  def execute(self, threshold, network, inputs):
    nn = neuralNetwork(network, inputs)
    self.pop = copy.deepcopy(self.generatePopulation(nn, network[0][0], network[0][1]))
    self.sort()
    gen = 0
    while self.pop[0].fitness < threshold:
      print('Generation: ', gen)
      print('Chromosome: ', self.pop[0].chr)
      print('Fitness: ', self.pop[0].fitness)
      self.sort()
      buffer = []
      buffer = copy.deepcopy(self.selection())
      for i in range(int(self.pop_size * self.crossoverRate)):
        parent1 = random.randint(0, self.pop_size * self.selectionRate)
        parent2 = random.randint(0, self.pop_size * self.selectionRate)
        offspring = self.crossover(self.pop[parent1], self.pop[parent2], nn, network[0][0], network[0][1])
        buffer.append(offspring)
      for i in range(int(self.pop_size * self.mutationRate)):
        ch = random.randint(0, self.pop_size * self.selectionRate)
        newCh = self.mutation(self.pop[ch], nn, network[0][0], network[0][1])
        buffer.append(newCh)
      self.pop = copy.deepcopy(buffer)
      gen += 1
    print('Generation: ', gen)
    print('Chromosome: ', self.pop[0].chr)
    print('Fitness: ', self.pop[0].fitness)
    return (make3D(self.pop[0].chr, network[0][0], network[0][1]), nn)
  def generatePopulation(self, nn, inputCount, outputCount):
    population=[]
    for i in range(self.pop_size):
      ch = generateChromosome(nn, inputCount, outputCount, self.trainingRate)
      population.append(ch)
    return population
  def sort(self):
    self.pop = sorted(self.pop, key=lambda x: x.fitness, reverse = True)
  def selection(self):
    buff = []
    for i in range(int(self.pop_size * self.selectionRate)):
      buff.append(self.pop[i])
    return buff
  def crossover(self, chromosome1, chromosome2, nn, inputCount, outputCount):
    length = len(chromosome1.chr)
    point1 = random.randint(0, length)
    point2 = random.randint(0, length)
    chr = []
    if point1 < point2:
      for i in range(point1):
        chr.append(chromosome1.chr[i])
      for i in range(point1, point2):
        chr.append(chromosome2.chr[i])
      for i in range(point2, length):
        chr.append(chromosome1.chr[i])
    else:
      for i in range(point2):
        chr.append(chromosome1.chr[i])
      for i in range(point2, point1):
        chr.append(chromosome2.chr[i])
      for i in range(point1, length):
        chr.append(chromosome1.chr[i])
    chromo = chromosome(chr, nn, inputCount, outputCount, self.trainingRate)
    return (chromo)
  def mutation(self, ch, nn, inputCount, outputCount):
    length = len(ch.chr)
    nop = random.randint(0, length) #no of points to mutate on
    for i in range (nop):
      point = random.randint(0, length - 1)
      ch.chr[point] = round(np.random.randn(), 4)
    chromo = chromosome(ch.chr, nn, inputCount, outputCount, self.trainingRate)
    return (chromo)
def getAccWeights(network, inputs, tRate):
  ga = geneticAlgorithm(tRate)
  (accWeights, nn) = ga.execute(100, network, inputs)
  return (accWeights, nn)

In [ ]:
def main():
  (network, inputs) = getUserInput()
  trainRate = 0.6
  (accWeights, neuralN) = getAccWeights(network, inputs, trainRate)
  print('Weights learnt on training data: ', accWeights)
  neuralN.setWeights(accWeights)
  acc = neuralN.accuracy(trainRate, 1)
  print('Accuracy of weights after learning weights on training data: ', acc, '%')

In [ ]:
main()